# Import ploonetide and other libraries

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from ploonetide import TidalSimulation
from ploonetide.utils import colorline
from ploonetide.utils.functions import mean2axis, find_moon_fate
from ploonetide.utils.constants import GYEAR, KYEAR, DAY, MSUN, AU

## Create TidalSimulation object for planet-moon system. The list of the most important parameters is printed out.

In [ ]:
simulation = TidalSimulation(
    system='planet-moon',
    star_mass=0.8,  # Solar masses
    star_radius=1.0,  # Solar radii
    star_rotperiod=10,  # days
    planet_mass=2,  # Jupiter masses
    planet_radius=1.3,  # Jupiter radii
    planet_orbperiod=20,  # days
    planet_rotperiod=0.6,  # days
    moon_eccentricty=0.0,
    moon_semimaxis=10,  # Roche radii
    planet_size_evolution=False,
    planet_internal_evolution=False,
    planet_core_dissipation=False,
)

## After building the TidalSimulation class instance, and before running the simulation, you still can modify any parameter in `dir(simulation)`.

In [ ]:
dir(simulation)
simulation.star_mass = 2.0  # Solar masses
simulation.sim_parameters

### Choose the total integration time and time-step

In [ ]:
integration_time = 1 * simulation.star_lifespan.value
timestep = 1000 * KYEAR

### Choose the integrator and run the simulation

In [ ]:
simulation.set_integration_method('rk4')
simulation.run(integration_time, timestep)

### Get the times and solutions

In [ ]:
times, solutions = simulation.history
nms = solutions[:, 0]
ops = solutions[:, 1]
nps = solutions[:, 2]
Tms = solutions[:, 3]
Ems = solutions[:, 4]
if simulation.moon_eccentricty != 0.0:
    ems = solutions[:, 5]

### Plot the solutions using matplotlib and a few helper functions from ploonetide 

In [ ]:
# COMPUTE SEMIMAJOR AXIS
ams = mean2axis(nms, simulation.planet_mass.to('kg').value, simulation.moon_mass.to('kg').value)
aps = mean2axis(nps, simulation.star_mass.to('kg').value, simulation.planet_mass.to('kg').value)

# ************************************************************
# FINDING ROUND-TRIP TIMES OR RUNAWAY TIMES
# ************************************************************
fate = find_moon_fate(times, ams, simulation.moon_roche_radius.value, simulation.planet_orbperiod.to('s').value,
                      simulation.planet_mass.to('kg').value, simulation.star_mass.to('kg').value)

labels = {'P': r'$\mathrm{Log_{10}}(P_\mathrm{orb})\mathrm{[d]}$',
          'Ms': r'$M_\bigstar[\mathrm{M_\odot}]$', 'Mp': r'$M_\mathrm{p}[\mathrm{M_{jup}}]$',
          't': r'$\mathrm{Log_{10}}(\mathrm{Time})\mathrm{[Gyr]}$'}

labelsize = 7.
markersize = 7.
ticksize = 6.

x = times / GYEAR
y = ams / simulation.moon_roche_radius.value
z = Tms

print(y)


fig = plt.figure(figsize=(5, 3.5))
ax = fig.add_subplot(1, 1, 1)
lc = colorline(x, y, z, cmap='jet')
cbar = fig.colorbar(lc, orientation='vertical', aspect=17, format="%.2f", pad=0.04)
cbar.set_label(label='Temperature [K]', size=7)
cbar.set_ticks(np.linspace(np.nanmin(z), np.nanmax(z), 9))
cbar.ax.tick_params(labelsize=ticksize)
# ax.axhline(am_roche / roche_lims, c="k", ls="--", lw=0.9, zorder=0.0, label="Roche limit")

ax.set_xlim(x.min(), x[fate.index] + 0.2)
ax.set_ylim(1.0, np.nanmax(y[np.isfinite(y)]) + 1)
ax.tick_params(axis='both', direction='in', labelsize=ticksize)

ax.set_xlabel('Time [Gyr]', fontsize=labelsize)
ax.set_ylabel(r'Moon semi-major axis [$a_\mathrm{Roche}$]', fontsize=labelsize)
# ax.legend(loc='upper right', fontsize=labelsize)
fig.savefig('migration.png', dpi=300, facecolor='w')


In [ ]:
simulation.star_luminosity.to('Lsun')